In [1]:
import re, subprocess, boto3, json, shlex, mysql, os, urllib, logging
import pandas as pd
import numpy as np
from s3path import S3Path
from pathlib import Path
from tqdm.notebook import tqdm
from packaging import version
pd.set_option("display.max_colwidth", 40)

In [2]:
from utils.utils import *

In [3]:
import glob

# Functions

## List FASTQs

In [4]:
# FASTQ reads/indices required for each workflow
fastq_map = {
    'MitoTracing': ['R1', 'R2'],
}

# Get fastq file paths on S3 for each file id
# Returns dictionary from id to s3 path
# Throws exception if FASTQs don't exist for any id
def get_fastqs(
    path: str, # path to directory containing FASTQ files
    fastq_file_ids: list, # FASTQ file ids needed for this run type (e.g. I1, R1, R2, etc.)

):
    fastq_map = dict()
    _, bucket, key, _, _ = urllib.parse.urlsplit(path)
    for fid in fastq_file_ids:
        files = get_s3_objects(
            bucket, key.lstrip("/"),
            re.compile(f"_{fid}_\d{{3}}.fastq.gz$")
        )
        try:
            assert files, f"AssertionError: Missing `{fid}` archives!"
            fastq_map[fid] = [os.path.join("s3://", bucket, str(f)) for f in files]
        except AssertionError as err:
            logging.warning("%s\n\t %s", err, path)
            return
    return fastq_map

In [5]:
def get_barcode_genomic_fastqs(samples):
    
    samples['fastq'] = np.empty((len(samples), 0)).tolist()
    fastqs = []
    for sample, row in samples.iterrows():
        S3_path = row['S3_path']
        
        # List FASTQ files
        fastq_file_ids = fastq_map['MitoTracing']        
        path_fastq = f"{S3_path}/FASTQ/"
        fastq = get_fastqs(path_fastq, fastq_file_ids)
        
        fastqs.append(fastq)
        
    samples['fastq'] = fastqs
    return samples

In [6]:
# Extract FASTQ sample name from list of files
# Note: FASTQ name is file name up to lane id (e.g. L001, L002, etc.)
def get_fastqs_name(fastqs):
    fastq_name_re = r".*/(.*)_S\d+_L\d{3}_[A-Za-z]\d_\d{3}.fastq.gz$"
    fastq_names = [re.match(fastq_name_re, x)[1] for x in fastqs]
    assert len(set(fastq_names)) == 1 # make sure all names are same
    return fastq_names[0]

## Get whitelist

In [7]:
# Get fastq file paths on S3 for each file id
# Returns dictionary from id to s3 path
# Throws exception if FASTQs don't exist for any id
def get_whitelist(
    path: str, # path to directory containing FASTQ files
):
    _, bucket, key, _, _ = urllib.parse.urlsplit(path)
    whitelist = get_s3_objects(
            bucket, key.lstrip("/"),
            re.compile(f".txt$")
        )
    whitelist = whitelist[0]
    try:
        whitelist = os.path.join("s3://", bucket, whitelist)
    except AssertionError as err:
        logging.warning("%s\n\t %s", err, path)
        return
    return whitelist

## DB queries

In [8]:
# Common query col: id, request_id, Sample
def get_sample_name(query, query_col, creds):
    
    user = creds['user']
    password = creds['password']
    
    try:
        table_sample_data = "peer_lab_db.sample_data"
        query = f"""
        SELECT {table_sample_data}.Sample
        FROM {table_sample_data}
        WHERE {table_sample_data}.{query_col}="{query}"
        """
        sample_names = []
        results = execute_query(query, user, password)
        for result in results:
            sample_names.append(result[0])
        return sample_names
    except Error as e:
        print(f"Error: {e}")
    
    
def get_aws_path(query, query_col, creds):
    
    user = creds['user']
    password = creds['password']
    
    try:
        table_sample_data = "peer_lab_db.sample_data"
        query = f"""
        SELECT {table_sample_data}.AWS_storage
        FROM {table_sample_data}
        WHERE {table_sample_data}.{query_col}="{query}"
        """
        sample_paths = []
        results = execute_query(query, user, password)
        for result in results:
            sample_paths.append(result[0])
        return sample_paths
    except Error as e:
        print(f"Error: {e}")
        
            
def get_sample_id(query, query_col, creds):
    
    user = creds['user']
    password = creds['password']
    
    try:
        table_sample_data = "peer_lab_db.sample_data"
        query = f"""
        SELECT {table_sample_data}.id
        FROM {table_sample_data}
        WHERE {table_sample_data}.{query_col}="{query}"
        """
        sample_ids = []
        results = execute_query(query, user, password)
        for result in results:
            sample_ids.append(result[0])
        return sample_ids
    except Error as e:
        print(f"Error: {e}")
        
def format_sample_aws(querys, query_col, creds):
    sample_names = []
    sample_paths = []
    sample_ids = []
    
    for query in querys:
        sample_names += get_sample_name(query, query_col, creds)
        sample_paths += get_aws_path(query, query_col, creds)
        sample_ids += get_sample_id(query, query_col, creds)
        
    sample_paths = [s.strip('/') for s in sample_paths] # remove trailing slash if exists
    
    samples = pd.DataFrame(
        dict(S3_path=sample_paths, Sample_ID=sample_ids),
        index=sample_names,
        dtype=str,
    )
    return samples

In [9]:
def get_genomeIndex_id(query, query_col, creds):
    user = creds['user']
    password = creds['password']
    
    try:
        table_sample_data = "peer_lab_db.sample_data"
        query = f"""
        SELECT {table_sample_data}.genomeIndex_id
        FROM {table_sample_data}
        WHERE {table_sample_data}.{query_col}="{query}"
        """
        
        results = execute_query(query, user, password)
        for result in results:
            genomeIndex_id = result[0]
            
        return genomeIndex_id
            
    except Error as e:
        print(f"Error: {e}")
        
def get_scTech_id(query, query_col, creds):
    user = creds['user']
    password = creds['password']
    
    try:
        table_sample_data = "peer_lab_db.genome_index"
        query = f"""
        SELECT {table_sample_data}.scTech_id
        FROM {table_sample_data}
        WHERE {table_sample_data}.{query_col}="{query}"
        """
        
        results = execute_query(query, user, password)
        for result in results:
            scTech_id = result[0]
            
        return scTech_id
            
    except Error as e:
        print(f"Error: {e}")
        
def get_index(query, query_col, creds):
    user = creds['user']
    password = creds['password']
    
    try:
        table_sample_data = "peer_lab_db.genome_index"
        query = f"""
        SELECT {table_sample_data}.gIndex
        FROM {table_sample_data}
        WHERE {table_sample_data}.{query_col}="{query}"
        """
        
        results = execute_query(query, user, password)
        for result in results:
            index = result[0]
            
        return index
            
    except Error as e:
        print(f"Error: {e}")
        
def get_assay(query, query_col, creds):
    user = creds['user']
    password = creds['password']
    
    try:
        table_sample_data = "peer_lab_db.sc_tech"
        query = f"""
        SELECT {table_sample_data}.Run_name
        FROM {table_sample_data}
        WHERE {table_sample_data}.{query_col}="{query}"
        """
        
        results = execute_query(query, user, password)
        for result in results:
            assay = result[0]
            
        return assay
            
    except Error as e:
        print(f"Error: {e}")
        
def get_barcode(query, query_col, creds):
    user = creds['user']
    password = creds['password']
    
    try:
        table_sample_data = "peer_lab_db.sc_tech"
        query = f"""
        SELECT {table_sample_data}.barcodes
        FROM {table_sample_data}
        WHERE {table_sample_data}.{query_col}="{query}"
        """
        
        results = execute_query(query, user, password)
        for result in results:
            barcode = result[0]
            
        return barcode
            
    except Error as e:
        print(f"Error: {e}")
        
        
def format_assay_barcode(samples, creds):
    
    # samples['genomeIndex_id'] = np.nan
    samples['index'] = np.nan
    # samples['scTech_id'] = np.nan
    samples['assay'] = np.nan
    samples['barcode'] = np.nan
    
    for sample, row in samples.iterrows():
        sample_id = row['Sample_ID']
        
        genomeIndex_id  = get_genomeIndex_id(sample_id, 'id', creds)
        index = get_index(genomeIndex_id, 'id', creds)
        scTech_id = get_scTech_id(genomeIndex_id, 'id', creds)
        assay = get_assay(scTech_id, 'id', creds)
        barcode = get_barcode(scTech_id, 'id', creds)
        
        # samples.loc[sample, 'genomeIndex_id'] = genomeIndex_id
        samples.loc[sample, 'index'] = index
        # samples.loc[sample, 'scTech_id'] = scTech_id
        samples.loc[sample, 'assay'] = assay
        samples.loc[sample, 'barcode'] = barcode

    return samples

In [10]:
def get_project_id(sample_id, creds):
    
    user = creds['user']
    password = creds['password']
    
    try:
        table_sample_data = "peer_lab_db.sample_data"
        table_project_data = "peer_lab_db.project_data"
        query = f"""
        SELECT {table_project_data}.projectName
        FROM {table_project_data}
        LEFT JOIN {table_sample_data}
        ON {table_project_data}.id = {table_sample_data}.projectData_id
        WHERE {table_sample_data}.id = {sample_id}
        """
        result = execute_query(query, user, password)[0][0]
        return result
    except Error as e:
        print(f"Error: {e}")

# Process Samples

## Setup

In [11]:
# Maps from .wdl name (prefix) to results dirname
results_dirs = {
    "MitoTracing": "mito-tracing-outs",
}

# Maps from .wdl name (prefix) to shell script
sh_files = {
    "MitoTracing": "submit-processing.sh",
}

In [12]:
# Location of docker files
common_docker_registry = "quay.io/hisplan"

prefix = "MitoTracing" # Workflow to run; also .wdl filename prefix
pipeline_type = prefix # field in *.labels.json
output_dirname = results_dirs[prefix]
template_prefix = 'template'

# If need to add comment, put here
comment = ""

In [13]:
# Locations of workflow-related directories and files
path_to_cromwell_secrets = f"{Path.home()}/.cromwell/cromwell-secrets.json" # CHANGE THIS
workflow_dir = glob.glob(f"{Path.home()}/scing/bin/*mito*")[0]
path_to_exec = sh_files[prefix]
config_dir = f"{workflow_dir}/configs"
path_to_options = glob.glob(f"{workflow_dir}/*.options.aws.json")[0]

# Other file locations
db_credentials_path = f"{Path.home()}/.config.json" # CHANGE THIS

In [14]:
# Set credentials based on SCRIdb CLI config file
with open(db_credentials_path) as f:
    creds = json.load(f)

# Excution

## Sample information

In [17]:
# Common query col: id, request_id, Sample
request_ids = ['BM-1244', ]

samples = format_sample_aws(request_ids, 'request_id', creds)
samples

,S3_path,Sample_ID
RA19_10_13_FACs_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3008
RA19_10_14_FACs_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3009
RA19_10_17_FACs_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3010
RA19_10_18_FACs_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3011
RA19_10_23_FACs_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3012
RA19_10_14_FACS_citric_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3013


In [18]:
samples = format_assay_barcode(samples, creds)
samples

,S3_path,Sample_ID,index,assay,barcode
RA19_10_13_FACs_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3008,GRCh38-3.0.0,CR,NaN
RA19_10_14_FACs_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3009,GRCh38-3.0.0,CR,NaN
RA19_10_17_FACs_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3010,GRCh38-3.0.0,CR,NaN
RA19_10_18_FACs_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3011,GRCh38-3.0.0,CR,NaN
RA19_10_23_FACs_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3012,GRCh38-3.0.0,CR,NaN
RA19_10_14_FACS_citric_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3013,GRCh38-3.0.0,CR,NaN


In [19]:
samples = get_barcode_genomic_fastqs(samples)
samples

,S3_path,Sample_ID,index,assay,barcode,fastq
RA19_10_13_FACs_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3008,GRCh38-3.0.0,CR,NaN,{'R1': ['s3://dp-lab-data/SCRI_Proje...
RA19_10_14_FACs_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3009,GRCh38-3.0.0,CR,NaN,{'R1': ['s3://dp-lab-data/SCRI_Proje...
RA19_10_17_FACs_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3010,GRCh38-3.0.0,CR,NaN,{'R1': ['s3://dp-lab-data/SCRI_Proje...
RA19_10_18_FACs_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3011,GRCh38-3.0.0,CR,NaN,{'R1': ['s3://dp-lab-data/SCRI_Proje...
RA19_10_23_FACs_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3012,GRCh38-3.0.0,CR,NaN,{'R1': ['s3://dp-lab-data/SCRI_Proje...
RA19_10_14_FACS_citric_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3013,GRCh38-3.0.0,CR,NaN,{'R1': ['s3://dp-lab-data/SCRI_Proje...


In [20]:
# This sample has already been processed
samples = samples[samples.index != 'RA19_10_13_FACs_MITO']
samples

,S3_path,Sample_ID,index,assay,barcode,fastq
RA19_10_14_FACs_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3009,GRCh38-3.0.0,CR,NaN,{'R1': ['s3://dp-lab-data/SCRI_Proje...
RA19_10_17_FACs_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3010,GRCh38-3.0.0,CR,NaN,{'R1': ['s3://dp-lab-data/SCRI_Proje...
RA19_10_18_FACs_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3011,GRCh38-3.0.0,CR,NaN,{'R1': ['s3://dp-lab-data/SCRI_Proje...
RA19_10_23_FACs_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3012,GRCh38-3.0.0,CR,NaN,{'R1': ['s3://dp-lab-data/SCRI_Proje...
RA19_10_14_FACS_citric_MITO,s3://dp-lab-data/SCRI_Projects/HTA/M...,3013,GRCh38-3.0.0,CR,NaN,{'R1': ['s3://dp-lab-data/SCRI_Proje...


## Make input file

In [21]:
# Load minimum inputs and labels fields from input template
with open(f"{config_dir}/{template_prefix}.inputs.json") as f:
    std_inputs_fields = list(json.load(f).keys())
    
# Annotate all samples with workflow inputs and labels
inputs = pd.DataFrame(index=samples.index, columns=std_inputs_fields,)
inputs

,MitoTracing.sampleName,MitoTracing.fastqName,MitoTracing.fastqR1,MitoTracing.fastqR2,MitoTracing.reference,MitoTracing.includeIntrons,MitoTracing.expectCells,MitoTracing.numCores,MitoTracing.memory,MitoTracing.whitelist,MitoTracing.dockerRegistry
RA19_10_14_FACs_MITO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RA19_10_17_FACs_MITO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RA19_10_18_FACs_MITO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RA19_10_23_FACs_MITO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RA19_10_14_FACS_citric_MITO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Default inputs
inputs[f"{prefix}.includeIntrons"] = False
inputs[f"{prefix}.expectCells"] = 5000
inputs[f"{prefix}.numCores"] = 16
inputs[f"{prefix}.memory"] = 128

inputs[f"{prefix}.dockerRegistry"] = common_docker_registry

inputs

,MitoTracing.sampleName,MitoTracing.fastqName,MitoTracing.fastqR1,MitoTracing.fastqR2,MitoTracing.reference,MitoTracing.includeIntrons,MitoTracing.expectCells,MitoTracing.numCores,MitoTracing.memory,MitoTracing.whitelist,MitoTracing.dockerRegistry
RA19_10_14_FACs_MITO,NaN,NaN,NaN,NaN,NaN,False,5000,16,128,NaN,quay.io/hisplan
RA19_10_17_FACs_MITO,NaN,NaN,NaN,NaN,NaN,False,5000,16,128,NaN,quay.io/hisplan
RA19_10_18_FACs_MITO,NaN,NaN,NaN,NaN,NaN,False,5000,16,128,NaN,quay.io/hisplan
RA19_10_23_FACs_MITO,NaN,NaN,NaN,NaN,NaN,False,5000,16,128,NaN,quay.io/hisplan
RA19_10_14_FACS_citric_MITO,NaN,NaN,NaN,NaN,NaN,False,5000,16,128,NaN,quay.io/hisplan


In [23]:
# Sample information
inputs[f"{prefix}.sampleName"] = samples.index.tolist()

inputs[f"{prefix}.fastqR1"] = samples['fastq'].str['R1']
inputs[f"{prefix}.fastqR2"] = samples['fastq'].str['R2']

for sample in inputs.index:
    fastqR1_name = get_fastqs_name(inputs.loc[sample, 'MitoTracing.fastqR1'])
    fastqR2_name = get_fastqs_name(inputs.loc[sample, 'MitoTracing.fastqR2'])
    
    assert(fastqR1_name == fastqR1_name)
    inputs.loc[sample, f"{prefix}.fastqName"] = fastqR1_name
    
    # Check the whitelist to make sure that barcodes are in nucleotide format
    inputs.loc[sample, f"{prefix}.whitelist"] = get_whitelist(samples.loc[sample, 'S3_path'])


inputs

,MitoTracing.sampleName,MitoTracing.fastqName,MitoTracing.fastqR1,MitoTracing.fastqR2,MitoTracing.reference,MitoTracing.includeIntrons,MitoTracing.expectCells,MitoTracing.numCores,MitoTracing.memory,MitoTracing.whitelist,MitoTracing.dockerRegistry
RA19_10_14_FACs_MITO,RA19_10_14_FACs_MITO,3009_RA19_10_14_FACs_MITO_IGO_12411_5,[s3://dp-lab-data/SCRI_Projects/HTA/...,[s3://dp-lab-data/SCRI_Projects/HTA/...,NaN,False,5000,16,128,s3://dp-lab-data/SCRI_Projects/HTA/M...,quay.io/hisplan
RA19_10_17_FACs_MITO,RA19_10_17_FACs_MITO,3010_RA19_10_17_FACs_MITO_IGO_12411_6,[s3://dp-lab-data/SCRI_Projects/HTA/...,[s3://dp-lab-data/SCRI_Projects/HTA/...,NaN,False,5000,16,128,s3://dp-lab-data/SCRI_Projects/HTA/M...,quay.io/hisplan
RA19_10_18_FACs_MITO,RA19_10_18_FACs_MITO,3011_RA19_10_18_FACs_MITO_IGO_12411_7,[s3://dp-lab-data/SCRI_Projects/HTA/...,[s3://dp-lab-data/SCRI_Projects/HTA/...,NaN,False,5000,16,128,s3://dp-lab-data/SCRI_Projects/HTA/M...,quay.io/hisplan
RA19_10_23_FACs_MITO,RA19_10_23_FACs_MITO,3012_RA19_10_23_FACs_MITO_IGO_12411_8,[s3://dp-lab-data/SCRI_Projects/HTA/...,[s3://dp-lab-data/SCRI_Projects/HTA/...,NaN,False,5000,16,128,s3://dp-lab-data/SCRI_Projects/HTA/M...,quay.io/hisplan
RA19_10_14_FACS_citric_MITO,RA19_10_14_FACS_citric_MITO,3013_RA19_10_14_FACS_citric_MITO_IGO...,[s3://dp-lab-data/SCRI_Projects/HTA/...,[s3://dp-lab-data/SCRI_Projects/HTA/...,NaN,False,5000,16,128,s3://dp-lab-data/SCRI_Projects/HTA/M...,quay.io/hisplan


In [24]:
# Reference created by Jaeyoung
cellRangerRefPkg="s3://dp-lab-data/SCRI_Projects/HTA/Mito_tracing/reference-packages/refdata-gex-mito-GRCh38-ensemble98.tar.gz"
mitoFastaRefPkg="s3://dp-lab-data/SCRI_Projects/HTA/Mito_tracing/reference-packages/mito-fasta-GRCh38.tar.gz"

reference = [dict(cellRangerRefPkg=cellRangerRefPkg,
                  mitoFastaRefPkg=mitoFastaRefPkg)] * len(inputs)
inputs[f"{prefix}.reference"] = reference

inputs

,MitoTracing.sampleName,MitoTracing.fastqName,MitoTracing.fastqR1,MitoTracing.fastqR2,MitoTracing.reference,MitoTracing.includeIntrons,MitoTracing.expectCells,MitoTracing.numCores,MitoTracing.memory,MitoTracing.whitelist,MitoTracing.dockerRegistry
RA19_10_14_FACs_MITO,RA19_10_14_FACs_MITO,3009_RA19_10_14_FACs_MITO_IGO_12411_5,[s3://dp-lab-data/SCRI_Projects/HTA/...,[s3://dp-lab-data/SCRI_Projects/HTA/...,{'cellRangerRefPkg': 's3://dp-lab-da...,False,5000,16,128,s3://dp-lab-data/SCRI_Projects/HTA/M...,quay.io/hisplan
RA19_10_17_FACs_MITO,RA19_10_17_FACs_MITO,3010_RA19_10_17_FACs_MITO_IGO_12411_6,[s3://dp-lab-data/SCRI_Projects/HTA/...,[s3://dp-lab-data/SCRI_Projects/HTA/...,{'cellRangerRefPkg': 's3://dp-lab-da...,False,5000,16,128,s3://dp-lab-data/SCRI_Projects/HTA/M...,quay.io/hisplan
RA19_10_18_FACs_MITO,RA19_10_18_FACs_MITO,3011_RA19_10_18_FACs_MITO_IGO_12411_7,[s3://dp-lab-data/SCRI_Projects/HTA/...,[s3://dp-lab-data/SCRI_Projects/HTA/...,{'cellRangerRefPkg': 's3://dp-lab-da...,False,5000,16,128,s3://dp-lab-data/SCRI_Projects/HTA/M...,quay.io/hisplan
RA19_10_23_FACs_MITO,RA19_10_23_FACs_MITO,3012_RA19_10_23_FACs_MITO_IGO_12411_8,[s3://dp-lab-data/SCRI_Projects/HTA/...,[s3://dp-lab-data/SCRI_Projects/HTA/...,{'cellRangerRefPkg': 's3://dp-lab-da...,False,5000,16,128,s3://dp-lab-data/SCRI_Projects/HTA/M...,quay.io/hisplan
RA19_10_14_FACS_citric_MITO,RA19_10_14_FACS_citric_MITO,3013_RA19_10_14_FACS_citric_MITO_IGO...,[s3://dp-lab-data/SCRI_Projects/HTA/...,[s3://dp-lab-data/SCRI_Projects/HTA/...,{'cellRangerRefPkg': 's3://dp-lab-da...,False,5000,16,128,s3://dp-lab-data/SCRI_Projects/HTA/M...,quay.io/hisplan


## Make label file

In [25]:
# Load minimum inputs and labels fields from labels template
with open(f"{config_dir}/{template_prefix}.labels.json") as f:
    std_labels_fields = list(json.load(f).keys())
    
# Annotate all samples with workflow labels
labels = pd.DataFrame(index=samples.index, columns=std_labels_fields,)
labels

,pipelineType,project,sample,owner,destination,transfer,comment
RA19_10_14_FACs_MITO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RA19_10_17_FACs_MITO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RA19_10_18_FACs_MITO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RA19_10_23_FACs_MITO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RA19_10_14_FACS_citric_MITO,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Annotate labels
labels["pipelineType"] = pipeline_type
labels["project"] = samples["Sample_ID"].apply(lambda x: get_project_id(x, creds))
labels["sample"] = labels.index
labels["owner"] = creds["user"]
labels["destination"] = samples['S3_path'] + "/" + output_dirname
labels["transfer"] = "-"
labels["comment"] = creds["user"]

labels

,pipelineType,project,sample,owner,destination,transfer,comment
RA19_10_14_FACs_MITO,MitoTracing,HTAN_CITEseq,RA19_10_14_FACs_MITO,sohailn,s3://dp-lab-data/SCRI_Projects/HTA/M...,-,sohailn
RA19_10_17_FACs_MITO,MitoTracing,HTAN_CITEseq,RA19_10_17_FACs_MITO,sohailn,s3://dp-lab-data/SCRI_Projects/HTA/M...,-,sohailn
RA19_10_18_FACs_MITO,MitoTracing,HTAN_CITEseq,RA19_10_18_FACs_MITO,sohailn,s3://dp-lab-data/SCRI_Projects/HTA/M...,-,sohailn
RA19_10_23_FACs_MITO,MitoTracing,HTAN_CITEseq,RA19_10_23_FACs_MITO,sohailn,s3://dp-lab-data/SCRI_Projects/HTA/M...,-,sohailn
RA19_10_14_FACS_citric_MITO,MitoTracing,HTAN_CITEseq,RA19_10_14_FACS_citric_MITO,sohailn,s3://dp-lab-data/SCRI_Projects/HTA/M...,-,sohailn


# Submit job

In [27]:
inputs

,MitoTracing.sampleName,MitoTracing.fastqName,MitoTracing.fastqR1,MitoTracing.fastqR2,MitoTracing.reference,MitoTracing.includeIntrons,MitoTracing.expectCells,MitoTracing.numCores,MitoTracing.memory,MitoTracing.whitelist,MitoTracing.dockerRegistry
RA19_10_14_FACs_MITO,RA19_10_14_FACs_MITO,3009_RA19_10_14_FACs_MITO_IGO_12411_5,[s3://dp-lab-data/SCRI_Projects/HTA/...,[s3://dp-lab-data/SCRI_Projects/HTA/...,{'cellRangerRefPkg': 's3://dp-lab-da...,False,5000,16,128,s3://dp-lab-data/SCRI_Projects/HTA/M...,quay.io/hisplan
RA19_10_17_FACs_MITO,RA19_10_17_FACs_MITO,3010_RA19_10_17_FACs_MITO_IGO_12411_6,[s3://dp-lab-data/SCRI_Projects/HTA/...,[s3://dp-lab-data/SCRI_Projects/HTA/...,{'cellRangerRefPkg': 's3://dp-lab-da...,False,5000,16,128,s3://dp-lab-data/SCRI_Projects/HTA/M...,quay.io/hisplan
RA19_10_18_FACs_MITO,RA19_10_18_FACs_MITO,3011_RA19_10_18_FACs_MITO_IGO_12411_7,[s3://dp-lab-data/SCRI_Projects/HTA/...,[s3://dp-lab-data/SCRI_Projects/HTA/...,{'cellRangerRefPkg': 's3://dp-lab-da...,False,5000,16,128,s3://dp-lab-data/SCRI_Projects/HTA/M...,quay.io/hisplan
RA19_10_23_FACs_MITO,RA19_10_23_FACs_MITO,3012_RA19_10_23_FACs_MITO_IGO_12411_8,[s3://dp-lab-data/SCRI_Projects/HTA/...,[s3://dp-lab-data/SCRI_Projects/HTA/...,{'cellRangerRefPkg': 's3://dp-lab-da...,False,5000,16,128,s3://dp-lab-data/SCRI_Projects/HTA/M...,quay.io/hisplan
RA19_10_14_FACS_citric_MITO,RA19_10_14_FACS_citric_MITO,3013_RA19_10_14_FACS_citric_MITO_IGO...,[s3://dp-lab-data/SCRI_Projects/HTA/...,[s3://dp-lab-data/SCRI_Projects/HTA/...,{'cellRangerRefPkg': 's3://dp-lab-da...,False,5000,16,128,s3://dp-lab-data/SCRI_Projects/HTA/M...,quay.io/hisplan


In [28]:
labels

,pipelineType,project,sample,owner,destination,transfer,comment
RA19_10_14_FACs_MITO,MitoTracing,HTAN_CITEseq,RA19_10_14_FACs_MITO,sohailn,s3://dp-lab-data/SCRI_Projects/HTA/M...,-,sohailn
RA19_10_17_FACs_MITO,MitoTracing,HTAN_CITEseq,RA19_10_17_FACs_MITO,sohailn,s3://dp-lab-data/SCRI_Projects/HTA/M...,-,sohailn
RA19_10_18_FACs_MITO,MitoTracing,HTAN_CITEseq,RA19_10_18_FACs_MITO,sohailn,s3://dp-lab-data/SCRI_Projects/HTA/M...,-,sohailn
RA19_10_23_FACs_MITO,MitoTracing,HTAN_CITEseq,RA19_10_23_FACs_MITO,sohailn,s3://dp-lab-data/SCRI_Projects/HTA/M...,-,sohailn
RA19_10_14_FACS_citric_MITO,MitoTracing,HTAN_CITEseq,RA19_10_14_FACS_citric_MITO,sohailn,s3://dp-lab-data/SCRI_Projects/HTA/M...,-,sohailn


In [29]:
import time

assert (std_inputs_fields == list(inputs.columns)) & (inputs.notna().values.all())
assert (std_labels_fields == list(labels.columns)) & (labels.notna().values.all())

stdouts = [] # to store all outputs
process = True

with tqdm(inputs.index) as t:

    for sample_name in t:

        # Write inputs and labels to file
        path_to_inputs = f"{config_dir}/{sample_name}_{prefix}.inputs.json"
        with open(path_to_inputs, "w") as f_inputs:
            json.dump(inputs.loc[sample_name].to_dict(), f_inputs, indent=4, cls=NpEncoder)

        path_to_labels = f"{config_dir}/{sample_name}_{prefix}.labels.json"
        with open(path_to_labels, "w") as f_labels:
            json.dump(labels.loc[sample_name].to_dict(), f_labels, indent=4, cls=NpEncoder)

        if process:
            stdouts.append(run(
                workflow_path = workflow_dir,
                execp = path_to_exec,
                secrets = path_to_cromwell_secrets,
                inputs = path_to_inputs,
                labels = path_to_labels,
                options = path_to_options,
            ))
            
        # Sometimes causes problems if too many samples are run at once
        time.sleep(20) # Delay for 20 seconds.

  0%|          | 0/5 [00:00<?, ?it/s]

In [30]:
print(path_to_inputs)
print(path_to_labels)

/Users/sohailn/scing/bin/wdl-mito-tracing/configs/RA19_10_14_FACS_citric_MITO_MitoTracing.inputs.json
/Users/sohailn/scing/bin/wdl-mito-tracing/configs/RA19_10_14_FACS_citric_MITO_MitoTracing.labels.json


In [31]:
stdouts

[{'args': ['/Users/sohailn/scing/bin/wdl-mito-tracing/submit-processing.sh',
   '-k',
   '/Users/sohailn/.cromwell/cromwell-secrets.json',
   '-i',
   '/Users/sohailn/scing/bin/wdl-mito-tracing/configs/RA19_10_14_FACs_MITO_MitoTracing.inputs.json',
   '-l',
   '/Users/sohailn/scing/bin/wdl-mito-tracing/configs/RA19_10_14_FACs_MITO_MitoTracing.labels.json',
   '-o',
   '/Users/sohailn/scing/bin/wdl-mito-tracing/MitoTracing.options.aws.json'],
  'returncode': 0,
  'stdout': '{"id":"bea801a8-dd25-489e-87c6-bb07e87d3548","status":"Submitted"}\n',
  'stderr': ''},
 {'args': ['/Users/sohailn/scing/bin/wdl-mito-tracing/submit-processing.sh',
   '-k',
   '/Users/sohailn/.cromwell/cromwell-secrets.json',
   '-i',
   '/Users/sohailn/scing/bin/wdl-mito-tracing/configs/RA19_10_17_FACs_MITO_MitoTracing.inputs.json',
   '-l',
   '/Users/sohailn/scing/bin/wdl-mito-tracing/configs/RA19_10_17_FACs_MITO_MitoTracing.labels.json',
   '-o',
   '/Users/sohailn/scing/bin/wdl-mito-tracing/MitoTracing.options.

In [32]:
print(labels['destination'].values)

['s3://dp-lab-data/SCRI_Projects/HTA/Mito_tracing/RA19_10_14_FACs_MITO/mito-tracing-outs'
 's3://dp-lab-data/SCRI_Projects/HTA/Mito_tracing/RA19_10_17_FACs_MITO/mito-tracing-outs'
 's3://dp-lab-data/SCRI_Projects/HTA/Mito_tracing/RA19_10_18_FACs_MITO/mito-tracing-outs'
 's3://dp-lab-data/SCRI_Projects/HTA/Mito_tracing/RA19_10_23_FACs_MITO/mito-tracing-outs'
 's3://dp-lab-data/SCRI_Projects/HTA/Mito_tracing/RA19_10_14_FACS_citric_MITO/mito-tracing-outs']
